In [5]:
import os
import pandas as pd
import transformers

In [6]:
btred= pd.read_csv('../Data/bitcoin_reddit_all.csv')

/var/folders/f3/g1154c6d5hb7y4_s6bzsyw8m0000gn/T/ipykernel_62936/1397401860.py:1: DtypeWarning: Columns (0,1,2,3,4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  btred= pd.read_csv('../Data/bitcoin_reddit_all.csv')


# Data Prep

In [7]:
btred.head()

,Unnamed: 0,datetime,date,author,subreddit,created_utc,score,controversiality,body
0,0,2014-06-26 16:49:41,2014-06-26,HonorConnor,AskReddit,1.403801e+09,162.0,0.0,How do you feel about Bitcoin? I don't really ...
1,1,2014-05-17 23:30:36,2014-05-17,Talvoren,AskReddit,1.400369e+09,75.0,0.0,[The guy who blew a huge portion of his and hi...
2,2,2014-12-18 04:54:48,2014-12-18,ninjoe87,Outdoors,1.418878e+09,-4.0,0.0,Because you are a damn liar.\n\nFrom my other ...
3,3,2014-04-30 00:24:16,2014-04-30,fathergrigori54,tf2,1.398817e+09,13.0,0.0,This was the bitcoin hat guy that ACTUALLY del...
4,4,2014-07-07 16:06:21,2014-07-07,Draber-Bien,starcraft,1.404749e+09,19.0,0.0,I'm sure this is *good* for bitcoin


Check how many records it is

In [8]:
len(btred.index)

4309244

## Date Filtering

This dataset and the twitter dataset we want to join it to have different min/max times. Since we will be joining these based on time, we can go ahead and drop any records that occured in this dataset either before or after the twitter data set. 

In [9]:
#import the other dataset
twit_df = pd.read_csv('../Data/twitter_data.csv', sep=';')

In [10]:
#define a function to output the min/max dates
def min_max_dates(df, column_name, source):
    df[column_name] = df[column_name].astype('datetime64[ns]')
    r = {'min': df[column_name].min(),
         'max': df[column_name].max()}
    
    print(f"{source} data earliest date \t:{r['min']}", '\n'
          f"{source} data most recent date \t:{r['max']}")
    return r

In [11]:
twitter_date_range = min_max_dates(twit_df, 'Date', 'Twitter')
reddit_date_range = min_max_dates(btred, 'datetime','Reddit')

Twitter data earliest date 	:2017-08-01 00:00:00 
Twitter data most recent date 	:2019-01-21 23:00:00
Reddit data earliest date 	:2009-05-08 13:54:29 
Reddit data most recent date 	:2019-12-31 23:58:38


In [12]:
btred_filtered = btred.drop(btred[btred['datetime']< twitter_date_range['min']].index)
btred_filtered = btred_filtered.drop(btred_filtered[btred_filtered['datetime'] > twitter_date_range['max']].index)


In [13]:
min_max_dates(btred_filtered, 'datetime','Reddit')

Reddit data earliest date 	:2017-08-01 00:00:33 
Reddit data most recent date 	:2019-01-21 22:57:35


{'min': Timestamp('2017-08-01 00:00:33'),
 'max': Timestamp('2019-01-21 22:57:35')}

In [14]:
bitcoin_data = btred_filtered
del btred_filtered

Now lets look at our data:

In [15]:
bitcoin_data.head(3)

,Unnamed: 0,datetime,date,author,subreddit,created_utc,score,controversiality,body
21497,I sell bitcoin from circle. I charge a little ...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35812,And then you are also assuming that every pers...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39497,I actually hated the doge meme but eventually ...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
bitcoin_data.tail(3)

,Unnamed: 0,datetime,date,author,subreddit,created_utc,score,controversiality,body
4308169,4192285,2019-01-19 22:00:33,2019-01-19,jakesonwu,btc,1.547935e+09,-1.0,1.0,[Who controls Bitcoin Core ?](https://medium.c...
4308170,4192286,2019-01-02 23:26:56,2019-01-02,shitpplsay,CryptoCurrency,1.546472e+09,-1.0,1.0,Should never ever have to or be able to tie a ...
4308172,4192288,2019-01-18 20:50:36,2019-01-18,whalecheetah,btc,1.547845e+09,-1.0,1.0,There are no two primary Bitcoin Forks. There ...


Looks like we have some data quality issues. Why do some of our rows not have any values in most of the columns? Lets dig a little deeper, and see if there is a large number of these.  

In [17]:
missing = len(bitcoin_data[bitcoin_data['datetime'].isnull() ==True])
not_missing = len(bitcoin_data[bitcoin_data['datetime'].isnull() ==False])
print(f"Has missing fields \t : {missing}")
print(f"No missing fields \t : {not_missing}")
print(f'{round(missing/(not_missing + missing), 2)*100}% of records have formatting issues.')
      

Has missing fields 	 : 65695
No missing fields 	 : 1705049
4.0% of records have formatting issues.


Given 4% have formatting issue that do not allow us to retrieve the time they were posted, we will go ahead and drop these records from our dataset.

In [18]:
bitcoin_data = bitcoin_data.drop(bitcoin_data[bitcoin_data['datetime'].isnull() ==True].index)

In [19]:
bitcoin_data.reset_index(inplace=True)
bitcoin_data.head()

,index,Unnamed: 0,datetime,date,author,subreddit,created_utc,score,controversiality,body
0,1758064,1642193,2017-08-31 06:06:41,2017-08-31,jstolfi,btc,1.504160e+09,2.0,0.0,&gt; Of course the devs force their plans on e...
1,1758065,1642194,2017-08-04 18:33:17,2017-08-04,zanetackett,btc,1.501872e+09,18.0,0.0,is that the case currently? Bitcoin seems to f...
2,1758066,1642195,2017-08-09 20:50:47,2017-08-09,csehszlovakze,MGTOW,1.502312e+09,3.0,0.0,he could move his ass to a non-extradition cou...
3,1758067,1642196,2017-08-27 00:09:45,2017-08-27,ohsnapitsjaycee,GCXRep,1.503793e+09,1.0,0.0,**#98**\n\nTRADE COMPLETED WITH: /u/crustymech...
4,1758068,1642197,2017-08-14 16:25:49,2017-08-14,dagfar69,brasil,1.502728e+09,2.0,0.0,"Ele explicou como receber o ""salário"" em bitco..."


## Comment text cleansing

Looking at the first few comments, it definetly appears we will need to do some processing on the text to clean it up before pushing through a sentiment analyzer.

A couple of items we should be sure to handle:
- urls
- special characters
- new lines
- foreign languages
- numbers (typically do not add context to the sentiment)

In [20]:
bitcoin_data['body'] = bitcoin_data['body'].replace({'[^\w\s.?!]':'',
                                         '\d':'',
                                         '\n':' ',
                                         'http.*\s':' ',
                                            }, regex=True)
bitcoin_data
# pd.Series(['foo', 'fuz', np.nan]).str.replace('f.', 'ba', regex=True)

,index,Unnamed: 0,datetime,date,author,subreddit,created_utc,score,controversiality,body
0,1758064,1642193,2017-08-31 06:06:41,2017-08-31,jstolfi,btc,1.504160e+09,2.0,0.0,gt Of course the devs force their plans on eve...
1,1758065,1642194,2017-08-04 18:33:17,2017-08-04,zanetackett,btc,1.501872e+09,18.0,0.0,is that the case currently? Bitcoin seems to f...
2,1758066,1642195,2017-08-09 20:50:47,2017-08-09,csehszlovakze,MGTOW,1.502312e+09,3.0,0.0,he could move his ass to a nonextradition coun...
3,1758067,1642196,2017-08-27 00:09:45,2017-08-27,ohsnapitsjaycee,GCXRep,1.503793e+09,1.0,0.0,TRADE COMPLETED WITH ucrustymech ITEMS TRAD...
4,1758068,1642197,2017-08-14 16:25:49,2017-08-14,dagfar69,brasil,1.502728e+09,2.0,0.0,Ele explicou como receber o salário em bitcoin...
...,...,...,...,...,...,...,...,...,...,...
1705044,4308167,4192283,2019-01-05 19:45:06,2019-01-05,accelerated_dragon,litecoin,1.546718e+09,-1.0,1.0,Visa does times more than bitcoin. The streng...
1705045,4308168,4192284,2019-01-02 20:33:42,2019-01-02,emreddit,CryptoCurrency,1.546461e+09,-1.0,0.0,why are you lying? If its Bitcoin will you buy...
1705046,4308169,4192285,2019-01-19 22:00:33,2019-01-19,jakesonwu,btc,1.547935e+09,-1.0,1.0,Who controls Bitcoin Core ?httpsmedium.comlopp...
1705047,4308170,4192286,2019-01-02 23:26:56,2019-01-02,shitpplsay,CryptoCurrency,1.546472e+09,-1.0,1.0,Should never ever have to or be able to tie a ...


Now to handle the non english items. To do this, we will use an external package.

In [21]:
import numpy as np


In [34]:
bitcoin_data_list = np.array_split(bitcoin_data, int(len(bitcoin_data.index)/10000))

In [35]:
len(bitcoin_data_list[0].index)

10030

In [23]:
from langdetect import detect



def lang_detect_df(row):
    try: 
        lang = detect(row['body'])
    except:
        lang = 'unknown'
    return lang
    

# bitcoin_data['lang'] = bitcoin_data.apply(lang_detect_df, axis =1)
bitcoin_data.head()

,index,Unnamed: 0,datetime,date,author,subreddit,created_utc,score,controversiality,body
0,1758064,1642193,2017-08-31 06:06:41,2017-08-31,jstolfi,btc,1.504160e+09,2.0,0.0,gt Of course the devs force their plans on eve...
1,1758065,1642194,2017-08-04 18:33:17,2017-08-04,zanetackett,btc,1.501872e+09,18.0,0.0,is that the case currently? Bitcoin seems to f...
2,1758066,1642195,2017-08-09 20:50:47,2017-08-09,csehszlovakze,MGTOW,1.502312e+09,3.0,0.0,he could move his ass to a nonextradition coun...
3,1758067,1642196,2017-08-27 00:09:45,2017-08-27,ohsnapitsjaycee,GCXRep,1.503793e+09,1.0,0.0,TRADE COMPLETED WITH ucrustymech ITEMS TRAD...
4,1758068,1642197,2017-08-14 16:25:49,2017-08-14,dagfar69,brasil,1.502728e+09,2.0,0.0,Ele explicou como receber o salário em bitcoin...


In [ ]:
# bitcoin_data_fin = pd.DataFrame()
for i in range(len(bitcoin_data_list[10:])):
    bitcoin_data_lang = bitcoin_data_list[i]
    bitcoin_data_lang['lang'] = bitcoin_data_lang.apply(lang_detect_df, axis =1)
    bitcoin_data_fin = pd.concat([bitcoin_data_fin,bitcoin_data_lang], ignore_index=True)
    print(f'chunk {i} of {len(bitcoin_data_list)} completed')

bitcoin_data_fin.reset_index(inplace=True)
bitcoin_data_fin.to_csv('../Data/reddit_with_lang.csv')

chunk 0 of 170 completed
chunk 1 of 170 completed
chunk 2 of 170 completed
chunk 3 of 170 completed
chunk 4 of 170 completed
chunk 5 of 170 completed
chunk 6 of 170 completed
chunk 7 of 170 completed
chunk 8 of 170 completed
chunk 9 of 170 completed
chunk 10 of 170 completed
chunk 11 of 170 completed
chunk 12 of 170 completed
chunk 13 of 170 completed


In [29]:
bitcoin_data_lang.loc[bitcoin_data_lang['lang'] != 'en']

,index,Unnamed: 0,datetime,date,author,subreddit,created_utc,score,controversiality,body,lang
9972,1829223,1713352,2017-08-06 03:07:24,2017-08-06,sw33tandslutty,redditamateurvids,1.501989e+09,1.0,0.0,Get my latest premade vid Video GC.,es
9984,1829235,1713364,2017-08-09 20:45:58,2017-08-09,matg0d,brasil,1.502312e+09,0.0,0.0,Evitar bancos faz grande parte dos motivos por...,pt
10026,1829277,1713406,2017-08-30 15:16:28,2017-08-30,TweetsInCommentsBot,Bitcoin,1.504106e+09,1.0,0.0,PeterSchiff,de


In [ ]:
bitcoin_data = bitcoin_data.drop(bitcoin_data[bitcoin_data['lang'] != 'en'].index)

# Sentiment Tagging

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["It was the best of times.", "t was the worst of times."]
sentiment_pipeline(data)

In [ ]:
data = ["I think it is humorous that miners are considered intelligent people by so many. "]
sentiment_pipeline(data)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment = SentimentIntensityAnalyzer()
text_1 = "I think it is humorous that miners are considered intelligent people by so many. "
text_2 =  "The pizza tastes terrible."
sent_1 = sentiment.polarity_scores(text_1)
sent_2 = sentiment.polarity_scores(text_2)
print("Sentiment of text 1:", sent_1)
print("Sentiment of text 2:", sent_2)